In [1]:
from bs4 import BeautifulSoup
import urllib2
import requests

In [2]:
source=urllib2.urlopen('http://myneta.info/ls2014/index.php?action=summary&subAction=candidates_analyzed&sort=candidate#summary')

In [3]:
tree= BeautifulSoup(source,"html.parser")

In [4]:
len(tree)

22

In [5]:
tree.title

<title>Total candidates analyzed by NEW:Loksabha 2014 Election</title>

In [6]:
all_links = tree.find_all('a')

In [7]:
table_pol = tree.findAll('table')[2]
rows = table_pol.findAll("tr")[2:]

In [16]:
# We build one dictionary per candidate #

def c_link(r):
    return r.find("a").get("href")
def name(r):
    return r.find("a").get_text()
def cols(r):
    return r.findAll("td")
def assets(r):
    col = cols(r)
    ass = col[6].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",","")
    return ass
def liab(r):
    col = cols(r)
    liabilities = int(col[7].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",",""))
    return liabilities

info_candidate = lambda r: [int(cols(r)[0].get_text()), r.find("a").get("href"),r.find("a").get_text(),
                            cols(r)[2].get_text(),cols(r)[3].get_text(),cols(r)[5].get_text(),
                            cols(r)[4].get_text(),assets(r), liab(r)]

title = ['id','url','name','district','party','education','nr_crime','assets','liabilities']
dict_candidates = [dict(zip(title,info_candidate(r))) for r in rows]

In [17]:
len(dict_candidates)

8204

In [90]:
# Now each candidate has an url - so we parse each of their pages and scrape them#
short_url = dict_candidates[1555]['url']
full_url = "http://myneta.info/ls2014/"+short_url
print full_url
page_candidate = requests.get(full_url)
c_soup = BeautifulSoup(page_candidate.text,"html.parser")

http://myneta.info/ls2014/candidate.php?candidate_id=2885


In [89]:
#### Personal info #########

personal = c_soup.findAll(attrs={"class": "grid_3 alpha"})[0]
full_name = personal.find("h2").get_text()
district1 = personal.find("h5").get_text()
grid2 = personal.findAll(attrs={"class":"grid_2 alpha"})
party_full = grid2[0].get_text()
age = int(grid2[2].get_text().split(":")[1])
address = grid2[3].get_text().split(":")[1] # Careful this one changes
self_profession = personal.find("p").get_text().split('\n')[0].split(":")[1]
spouse_profession = personal.find("p").get_text().split('\n')[1].split(":")[1]
#print profession
#print spouse_profession
#print address
#print district1, party_full, age

########## Assets  & Criminality ###########
all_tables = c_soup.findAll("table")
print len(all_tables)
table_inc = all_tables[0]
table_crim = all_tables[1]
table_assets1 = all_tables[2]
table_assets2 = all_tables[3]
table_liab = all_tables[4]
#print table_inc
print all_tables


9
[<table cellpadding="0" cellspacing="0" id="table3" width="80%">
<tr style="background-color:#336633;color:white;" valign="top"><th colspan="3">Other Elections</th></tr>
<tr style="background-color:#336633;color:white;" valign="top"><th>Declaration in</th><th>Declared Assets</th><th>Declared Cases</th></tr>
<tr><td colspan="3"><a href="/compare_profile.php?group_id=FmV7ueoc9J52iv3Mwa">Click here for more details</a></td></tr></table>, <table border="1" width="100%">
<tr><th>Relation Type</th><th>PAN Given</th><th>Financial Year</th><th>Total Income Shown in ITR</th></tr>
<tr><td>self</td><td>Y</td><td>2012 - 2013</td><td><b>Rs 19,99,890</b><span style="color:black;font-size:60%;text-decoration:none;white-space:nowrap"> ~ 19 Lacs+</span></td></tr><tr><td>spouse</td><td>Y</td><td>2012 - 2013</td><td><b>Rs 4,76,400</b><span style="color:black;font-size:60%;text-decoration:none;white-space:nowrap"> ~ 4 Lacs+</span></td></tr><tr><td>dependent1</td><td>N</td><td>None</td><td><b>Rs 0</b><sp